In [17]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import random

# Number of locations and days
num_locations = 50
days = ['M', 'T', 'W', 'R', 'F']
num_days = len(days)

# Truck types and their capacities and costs
truck_types = [15, 10]   # capacities in tons
truck_costs = [150, 120] # costs in dollars

# Generate random weekly weight requirements between 10 and 50 tons for each location
W = [random.randint(10, 50) for _ in range(num_locations)]

# Initialize the model
model = gp.Model("TruckDeliveryOptimization")

# Decision variables
x = model.addVars(num_locations, num_days, len(truck_types), vtype=GRB.CONTINUOUS, name="x")  # Amount of weight delivered
z = model.addVars(num_days, len(truck_types), ub=15, vtype=GRB.INTEGER, name="z")  # Number of trucks used
y = model.addVars(num_locations, 9, vtype=GRB.BINARY, name="y")  # Delivery schedule
min_trucks = model.addVar(vtype=GRB.INTEGER, name="min_trucks")
min_trucks1 = model.addVar(vtype=GRB.INTEGER, name="min_trucks1")

# Define min_trucks constraints
for d in range(num_days):
    model.addConstr(min_trucks >= z[d, 0] + z[d, 1], name=f"min_trucks_lb_{d}")
    model.addConstr(min_trucks1 >= -(z[d, 0] - z[d, 1]), name=f"min_trucks_ub_{d}")
    model.addConstr(min_trucks1 >= (z[d, 0] - z[d, 1]), name=f"min_trucks_ub_{d}")

# Set objectives
model.setObjectiveN(sum(truck_costs[t] * z[d, t] 
                        for d in range(num_days) 
                        for t in range(len(truck_types))), priority=1, index=0, weight=1) 
model.setObjectiveN(min_trucks, priority=3, index=1, weight=1)
# model.setObjectiveN(min_trucks1, priority=2, index=2, weight=1)

# Constraints
# 1. Total weekly weight constraint
for i in range(num_locations):
    model.addConstr(gp.quicksum(x[i, d, t]
                                for d in range(num_days)
                                for t in range(len(truck_types))) >= W[i],
                    name=f"weekly_weight_{i}")

# 2. Schedule selection constraint (only one type of delivery schedule per location)
for i in range(num_locations):
    model.addConstr(gp.quicksum(y[i, s] for s in range(9)) == 1, name=f"schedule_selection_{i}")

# 3. Delivery day constraints based on specific patterns
patterns = {
    0: ['M', 'T', 'W', 'R', 'F'],  # Five times a week
    1: ['M'],                      # Once a week
    2: ['M', 'R'],                 # Twice a week
    3: ['M', 'W', 'F'],            # Thrice a week
    4: ['T'],
    5: ['W'],
    6: ['R'],
    7: ['F'],
    8: ['T', 'F']
}

# Map day indices to patterns
day_indices = {day: i for i, day in enumerate(days)}

# Indicator constraints for each pattern
for i in range(num_locations):
    for s in range(9):
        model.addGenConstrIndicator(y[i, s], 1,
                                    gp.quicksum(x[i, day_indices[d], t] 
                                                for d in patterns[s] for t in range(len(truck_types))) == gp.quicksum(x[i, d, t]
                                                for d in range(num_days)
                                                for t in range(len(truck_types))),
                                    name=f"pattern_{i}_{s}_{d}")
        for d in patterns[s]:
            temp = day_indices[patterns[s][0]]
            model.addGenConstrIndicator(y[i, s], 1,
                                        gp.quicksum(x[i, day_indices[d], t]
                                                    for t in range(len(truck_types))) == gp.quicksum(x[i, temp, t]
                                                                                                    for t in range(len(truck_types))),
                                        name=f"pattern_{i}_{s}_{d}")

# 4. Ensure that the total weight delivered by trucks does not exceed their capacity
for d in range(num_days):
    for t in range(len(truck_types)):
        model.addConstr(gp.quicksum(x[i, d, t] for i in range(num_locations)) <= truck_types[t] * z[d, t],
                        name=f"capacity_{d}_{t}")

# Optimize the model
model.optimize()

# Export results to Excel
if model.status == GRB.OPTIMAL:
    # Create DataFrames for Excel output
    summary_data = []
    details_data = []
    trucks_used_data = []
    patterns_selected = []
    truck_assignments_data = []

    # Track truck usage and assignments
    truck_assignments = {d: {t: [] for t in range(len(truck_types))} for d in range(num_days)}

    for i in range(num_locations):
        for d in range(num_days):
            for t in range(len(truck_types)):
                if x[i, d, t].x > 0.5:
                    summary_data.append([f"Location {i+1}", days[d], truck_types[t], x[i, d, t].x, W[i]])
                    details_data.append([f"Location {i+1}", days[d], f"Truck Type {truck_types[t]} tons", x[i, d, t].x])
                    truck_assignments[d][t].append(f"Location {i+1} ({x[i, d, t].x} tons)")

    for d in range(num_days):
        for t in range(len(truck_types)):
            trucks_used_data.append([days[d], f"Truck Type {truck_types[t]} tons", z[d, t].x])
            for assignment in truck_assignments[d][t]:
                truck_assignments_data.append([days[d], f"Truck Type {truck_types[t]} tons", assignment])

    for i in range(num_locations):
        for s in range(9):
            if y[i, s].x > 0.5:
                patterns_selected.append([f"Location {i+1}", patterns[s]])

    summary_df = pd.DataFrame(summary_data, columns=["Location", "Day", "Truck Type", "Weight Delivered (tons)", "Requirement (Tons)"])
    details_df = pd.DataFrame(details_data, columns=["Location", "Day", "Truck Type", "Weight Delivered (tons)"])
    trucks_used_df = pd.DataFrame(trucks_used_data, columns=["Day", "Truck Type", "Number of Trucks Used"])
    patterns_df = pd.DataFrame(patterns_selected, columns=["Location", "Pattern"])
    truck_assignments_df = pd.DataFrame(truck_assignments_data, columns=["Day", "Truck Type", "Location Assignment"])

    with pd.ExcelWriter("truck_delivery_optimization_results.xlsx") as writer:
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        details_df.to_excel(writer, sheet_name="Details", index=False)
        trucks_used_df.to_excel(writer, sheet_name="Trucks Used", index=False)
        patterns_df.to_excel(writer, sheet_name="Patterns Used", index=False)
        truck_assignments_df.to_excel(writer, sheet_name="Truck Assignments", index=False)

    print("Results exported to delivery_optimization_results1.xlsx")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 125 rows, 962 columns and 1505 nonzeros
Model fingerprint: 0x6dc52d44
Model has 1300 general constraints
Variable types: 500 continuous, 462 integer (450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 5e+01]
  GenCon coe range [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 2250 rows and 1050 col

In [15]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import random

# Number of locations and days
num_locations = 20
days = ['M', 'T', 'W', 'R', 'F']
num_days = len(days)

# Truck types and their capacities and costs
truck_types = [15, 10]   # capacities in tons
truck_costs = [150, 120] # costs in dollars

# Generate random weekly weight requirements between 10 and 50 tons for each location
W = [random.randint(10, 50) for _ in range(num_locations)]

# Initialize the model
model = gp.Model("TruckDeliveryOptimization")

# Decision variables
x = model.addVars(num_locations, num_days, len(truck_types), vtype=GRB.CONTINUOUS, name="x")  # Amount of weight delivered
z = model.addVars(num_days, len(truck_types), vtype=GRB.INTEGER, name="z")  # Number of trucks used
y = model.addVars(num_locations, 9, vtype=GRB.BINARY, name="y")  # Delivery schedule
assign = model.addVars(num_days, len(truck_types), num_locations, 15, vtype=GRB.BINARY, name="assign") # Truck assignments

min_trucks = model.addVar(vtype=GRB.INTEGER, name="min_trucks")

# Define min_trucks constraints
for d in range(num_days):
    model.addConstr(min_trucks >= z[d, 0] + z[d, 1], name=f"min_trucks_lb_{d}")

# Set objectives
model.setObjectiveN(sum(truck_costs[t] * z[d, t] 
                        for d in range(num_days) 
                        for t in range(len(truck_types))), priority=1, index=0, weight=1)
model.setObjectiveN(min_trucks, priority=2, index=1, weight=1)

# Constraints
# 1. Total weekly weight constraint
for i in range(num_locations):
    model.addConstr(gp.quicksum(x[i, d, t]
                                for d in range(num_days)
                                for t in range(len(truck_types))) >= W[i],
                    name=f"weekly_weight_{i}")

# 2. Schedule selection constraint (only one type of delivery schedule per location)
for i in range(num_locations):
    model.addConstr(gp.quicksum(y[i, s] for s in range(9)) == 1, name=f"schedule_selection_{i}")

# 3. Delivery day constraints based on specific patterns
patterns = {
    0: ['M', 'T', 'W', 'R', 'F'],  # Five times a week
    1: ['M'],                      # Once a week
    2: ['M', 'R'],                 # Twice a week
    3: ['M', 'W', 'F'],            # Thrice a week
    4: ['T'],
    5: ['W'],
    6: ['R'],
    7: ['F'],
    8: ['T', 'F']
}

# Map day indices to patterns
day_indices = {day: i for i, day in enumerate(days)}

# Indicator constraints for each pattern
for i in range(num_locations):
    for s in range(9):
        model.addGenConstrIndicator(y[i, s], 1,
                                    gp.quicksum(x[i, day_indices[d], t] 
                                                for d in patterns[s] for t in range(len(truck_types))) == gp.quicksum(x[i, d, t]
                                                for d in range(num_days)
                                                for t in range(len(truck_types))),
                                    name=f"pattern_{i}_{s}_{d}")
        for d in patterns[s]:
            temp = day_indices[patterns[s][0]]
            model.addGenConstrIndicator(y[i, s], 1,
                                        gp.quicksum(x[i, day_indices[d], t]
                                                    for t in range(len(truck_types))) == gp.quicksum(x[i, temp, t]
                                                                                                    for t in range(len(truck_types))),
                                        name=f"pattern_{i}_{s}_{d}")

# 4. Ensure that the total weight delivered by trucks does not exceed their capacity
for d in range(num_days):
    for t in range(len(truck_types)):
        model.addConstr(gp.quicksum(x[i, d, t] for i in range(num_locations)) <= truck_types[t] * z[d, t],
                        name=f"capacity_{d}_{t}")

# 5. Ensure consistency between truck usage and assignments
for d in range(num_days):
    for t in range(len(truck_types)):
        model.addConstr(z[d, t] == gp.quicksum(assign[d, t, i, k] for i in range(num_locations) for k in range(15)),
                        name=f"truck_usage_{d}_{t}")

# 6. Link truck assignments to weight deliveries
for i in range(num_locations):
    for d in range(num_days):
        for t in range(len(truck_types)):
            model.addConstr(x[i, d, t] == gp.quicksum(assign[d, t, i, k] * truck_types[t] for k in range(15)),
                            name=f"weight_delivery_{i}_{d}_{t}")

# 7. Ensure that the total weight assigned to a truck does not exceed its capacity
for d in range(num_days):
    for t in range(len(truck_types)):
        for k in range(15):
            model.addConstr(gp.quicksum(assign[d, t, i, k] * W[i] for i in range(num_locations)) <= truck_types[t],
                            name=f"truck_capacity_{d}_{t}_{k}")

# Optimize the model
model.optimize()

# Export results to Excel
if model.status == GRB.OPTIMAL:
    # Create DataFrames for Excel output
    summary_data = []
    details_data = []
    trucks_used_data = []
    patterns_selected = []
    truck_assignment_data = []

    for i in range(num_locations):
        for d in range(num_days):
            for t in range(len(truck_types)):
                if x[i, d, t].x > 0.5:
                    summary_data.append([f"Location {i+1}", days[d], truck_types[t], x[i, d, t].x, W[i]])
                    details_data.append([f"Location {i+1}", days[d], f"Truck Type {truck_types[t]} tons", x[i, d, t].x])

    for d in range(num_days):
        for t in range(len(truck_types)):
            trucks_used_data.append([days[d], f"Truck Type {truck_types[t]} tons", z[d, t].x])
            
            for i in range(num_locations):
                for k in range(15):
                    if assign[d, t, i, k].x > 0.5:
                        truck_assignment_data.append([f"Truck {k+1} ({truck_types[t]} tons)", days[d], f"Location {i+1}", x[i, d, t].x])

    for i in range(num_locations):
        for s in range(9):
            if y[i,s].x > 0.5:
                patterns_selected.append([f"Location {i+1}", patterns[s]])

    summary_df = pd.DataFrame(summary_data, columns=["Location", "Day", "Truck Type", "Weight Delivered (tons)", "Requirement (Tons)"])
    details_df = pd.DataFrame(details_data, columns=["Location", "Day", "Truck Type", "Weight Delivered (tons)"])
    trucks_used_df = pd.DataFrame(trucks_used_data, columns=["Day", "Truck Type", "Number of Trucks Used"])
    patterns_df = pd.DataFrame(patterns_selected, columns=["Location", "Pattern"])
    truck_assignment_df = pd.DataFrame(truck_assignment_data, columns=["Truck", "Day", "Location", "Weight Delivered (tons)"])

    with pd.ExcelWriter("delivery_optimization_results.xlsx") as writer:
        summary_df.to_excel(writer, sheet_name="Summary", index=False)
        details_df.to_excel(writer, sheet_name="Details", index=False)
        trucks_used_df.to_excel(writer, sheet_name="Trucks Used", index=False)
        patterns_df.to_excel(writer, sheet_name="Patterns Used", index=False)
        truck_assignment_df.to_excel(writer, sheet_name="Truck Assignment", index=False)

    print("Results exported to truck_delivery_optimization_results.xlsx")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information